In [41]:
import numpy as np
import pandas as pd
import string
import re

In [42]:
#读取npz格式的数据 先处理tag数据 因为数据小一点快一些
data=np.load('dfdata.npz',allow_pickle=True)
body=data['body']
tag=data['tag']

In [43]:
body_num=3000#取词频最高的300个body词
tag_num=300#取词频最高的300个tag词
tag=tag[0:10000]
tag[0:5]#观察一下tag的形式

array(['<front-end><syntax>',
       '<functions><performance-tuning><pattern-matching><filtering><function-comparison>',
       '<cdf-format>', '<topology><graphics>', '<graphics><plotting>'],
      dtype=object)

In [44]:
def remove_tag(sentence):#tag不需要进行单词还原是因为平台会自动完成tagrec部分 推荐正确的词形
    sentence=sentence.lower()#所有字母小写
    del_estr = string.punctuation + string.digits+string.whitespace  #去除ASCII，标点符号，数字
    del_estr=del_estr.replace('-','')#保留- 因为tag里有很多-有用
    replace = " "*len(del_estr)
    tran_tab = str.maketrans(del_estr, replace)
    sentence = sentence.translate(tran_tab)#完成上述去除标点符号的功能
    words=sentence.split(' ')#根据空格把句子分隔成单词
    while '' in words:
        words.remove('')#去除为空格的空值
    return words

In [45]:
tag2=[]#tag2为经过预处理后的词袋
for tagi in tag:
    tag2.append(remove_tag(tagi))
tag3=[]#tag3为所有的tag之和，用于统计词频
for tagi in tag2:
    tag3=tag3+tagi
len(tag3)

20805

In [46]:
import codecs
import collections
from operator import itemgetter
counter = collections.Counter()# 统计单词出现的频率
for word in tag3:
    counter[word] += 1#词频字典
# 按照词频顺序对单词进行排序
sorted_word_to_cnt = sorted(counter.items(), key=itemgetter(1), reverse=True)
sorted_words = [x[0] for x in sorted_word_to_cnt]
import codecs
import sys
vocab = sorted_words[:tag_num] #只选取前tag_num个tag词汇
vocab = ["<unknown_tag>"] + vocab#再加上一个unknown词 用来表示样本里低频词汇
# 读取词汇表，并建立词汇到单词编号的映射
word_to_id = {k: v for (k, v) in zip(vocab, range(len(vocab)))}
# 如果出现了被删除的低频词，则替换为"<unk>"
def get_id(word):
    return word_to_id[word] if word in word_to_id else word_to_id["<unknown_tag>"]

In [47]:
tag5=[]#tag5是最终数值化表示的tag集合 
for word in tag2:
    tag4=[]#tag4是每一个post对应的tag集合里的一个单词
    for w in word: 
        tag4=tag4+[get_id(w)]
    tag5=tag5+[tag4]
tag6=np.array(tag5)#tag6是array化的tag集合 作为代码的输入数据之一
len(tag6)

10000

In [48]:
body=body[0:10000]
body[0:1]#观察一下tag的形式

array(['<p>From the front end, <code>\\[InvisibleApplication]</code> can be entered as <kbd>Esc</kbd> <kbd>@</kbd> <kbd>Esc</kbd>, and is an invisible operator for <code>@</code>!. By an unfortunate combination of key-presses (there may have been a cat involved), this crept up in my code and I spent a great deal of time trying to figure out why in the world <code>f x</code> was being interpreted as <code>f[x]</code>. Example:</p>\n\n<p><img src="https://i.stack.imgur.com/2Hxll.png" alt="enter image description here"></p>\n\n<p>Now there is no way I could\'ve spotted this visually. The <code>*Form</code>s weren\'t of much help either. If you\'re careful enough, you can see an invisible character between <code>f</code> and <code>x</code> if you move your cursor across the expression. Eventually, I found this out only by looking at the contents of the cell. </p>\n\n<p>There\'s also <code>\\[InvisibleSpace]</code>, <code>\\[InvisibleComma]</code> and <code>\\[ImplicitPlus]</code>, which ar

In [49]:
#import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
wnl = WordNetLemmatizer() 
sw=stopwords.words('english')#这两步都是在循环前调用为了加快速度
def remove_body(sentence):
    cleanr = re.compile('<.*?>')#去除html标签
    sentence = re.sub(cleanr, ' ', sentence)#去除html标签
    sentence=sentence.lower()#字母小写化
    del_estr = string.punctuation + string.digits+string.whitespace  # 去除ASCII 标点符号，数字
    replace = " "*len(del_estr)
    tran_tab = str.maketrans(del_estr, replace)
    sentence = sentence.translate(tran_tab)#完成上述去除标点符号的功能
    #去除stopwords 并把句子转换成单词的列表
    words= [w for w in sentence.split(' ') if w not in sw]
    while '' in words:
        words.remove('')#去除list里的空值
    while 'n' in words:
        words.remove('n')#去除list里的p
    while 'f' in words:
        words.remove('f')#去除list里的f
    while 'x' in words:
        words.remove('x')#去除list里的x
    #不知道invisible之类的需要删除吗
    record=[]#这个循环在做词形还原
    for i in words:
        record=record+[wnl.lemmatize(i)]
    return record

In [50]:
body2=[]#body2用于对每条body完成刚刚的数据预处理操作
for bodyi in body:
    body2.append(remove_body(bodyi))
body3=[]#body3是汇总全部的body单词 用于词频统计
for bodyi in body2:
    body3=body3+bodyi
len(body2)

10000

In [51]:
import codecs
import collections
from operator import itemgetter
# 统计单词出现的频率
counter2 = collections.Counter()
for word in body3:
    counter2[word] += 1#词频字典
# 按照词频顺序对单词进行排序
sorted_word_to_cnt2 = sorted(counter2.items(), key=itemgetter(1), reverse=True)
sorted_words2 = [x[0] for x in sorted_word_to_cnt2]
import codecs
import sys
# 读取词汇表，并建立词汇到单词编号的映射。
vocab2 = sorted_words2[:body_num] #只选取前body_num个body词汇
vocab2 = ["unknown"] + vocab2#再加上一个unknown词 用来表示样本里低频词汇
word_to_id2 = {k: v for (k, v) in zip(vocab2, range(len(vocab2)))}
# 如果出现了被删除的低频词，则替换为"<unk>"
def get_id2(word):
    return word_to_id2[word] if word in word_to_id2 else word_to_id2["unknown"]

In [52]:
body5=[]#body5是所有body的数值表示
for word in body2:
    body4=[]#body4是每个body的数值表示
    for w in word: 
        body4=body4+[get_id2(w)]
    body5=body5+[body4]
body6=np.array(body5)
len(body6)#看一眼长度有没有问题

10000

In [53]:
#保存为data.npz文件
brs=body6#contextual
ms=tag6#no useful
sfs=tag6#tags 
np.savez('data.npz',brs= brs, ms = ms,sfs=ms,allow_pickle=True)
data=np.load('data.npz',allow_pickle=True)#读取一下试试看
len(data['brs'])#再看一看长度有没有问题

10000

In [54]:
#为了后续的分析 也把tag和body里单词与数值对应的字典保存下来
np.save('word_to_id.npy', word_to_id,allow_pickle=True) 
np.save('word_to_id2.npy', word_to_id2,allow_pickle=True) 